In [1]:
import pickle
import os
from dotenv import load_dotenv
from pymilvus import utility, connections, FieldSchema, CollectionSchema, DataType, Collection

In [2]:
filename="TDS_8_30_2023"
        
with open(f'{filename}.pkl', 'rb') as f:
    data_batch = pickle.load(f)

In [17]:
load_dotenv()

zilliz_uri = os.getenv("ZILLIZ_URI")
zilliz_token = os.getenv("ZILLIZ_TOKEN")

In [16]:
connections.connect(
    uri= zilliz_uri,
    token= zilliz_token
)

In [26]:
DIMENSION=384
COLLECTION_NAME="tds_articles"

In [61]:
# utility.drop_collection(COLLECTION_NAME)

In [62]:
"""
"id": f"{article_url}+{i}",
"article_url": article_url,
"title": title,
"subtitle": subtitle,
"claps": claps,
"responses": responses,
"reading_time": reading_time,
"publication": publication,
"date": date,
"paragraph": paragraph.text,
"embedding": embedding
"""
fields = [
    FieldSchema(name='id', dtype=DataType.VARCHAR, max_length=200, is_primary=True),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]

In [63]:
data_batch[-1]

{'article_url': 'https://towardsdatascience.com/how-to-one-way-anova-by-hand-4c19e2a61a8c',
 'title': 'How To: One-Way ANOVA by Hand',
 'subtitle': 'Learn how to perform a one-way Analysis Of VAriance (ANOVA) by hand to…',
 'claps': 30,
 'responses': 0,
 'reading_time': 6,
 'publication': 'Towards Data Science',
 'date': '2023-08-30',
 'paragraph': 'Note that the ANOVA requires some assumptions (i.e., independence, equality of variances and normality). The aim of this post is to illustrate how to do an ANOVA by hand and not how to verify these assumptions, so we suppose they are met without any verification. See how to test these assumptions in R if you are interested.',
 'embedding': array([ 1.02438033e-02,  2.63246838e-02,  3.95597294e-02, -3.51585522e-02,
        -6.38489425e-02,  3.91995534e-03, -6.00639731e-02,  1.37220668e-02,
        -6.36992082e-02, -1.08909970e-02,  3.19623463e-02, -2.59741675e-03,
        -4.15538903e-03, -4.19805311e-02, -3.53036746e-02, -2.51680985e-02,
   

In [64]:
schema = CollectionSchema(fields=fields, enable_dynamic_field=True)
collection = Collection(name=COLLECTION_NAME, schema=schema)

In [65]:
index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
collection.create_index(field_name="embedding", index_params=index_params)
collection.load()

In [66]:
for data in data_batch:
    collection.insert([data])

In [67]:
collection.flush()

In [68]:
collection.num_entities

24394